In [1]:
import ast
import json

import pandas as pd

In [2]:
# Read the CSV
df = pd.read_csv("data/output/processed_ingredients.csv")
# Parse the JSON-style ingredient lists
df["ingredients_normalized"] = df["ingredients_normalized"].apply(json.loads)
# Ensure they're lists
df["ingredients_list"] = df["ingredients_normalized"].apply(
    lambda x: ast.literal_eval(str(x)) if isinstance(x, str) else x
)

In [3]:
df["ingredients_list"].head(5)

0    [brown sugar, evaporated milk, vanilla, pecan,...
1    [chipped beef, chicken breast, cream of mushro...
2    [corn, cream cheese, butter, garlic powder, sa...
3    [chicken, chicken gravy, mushroom soup, stuffi...
4    [peanut butter, graham cracker crumb, butter, ...
Name: ingredients_list, dtype: object

In [4]:
# Flatten all lists and deduplicate
ingredients_set = set()
for sublist in df["ingredients_list"]:
    for item in sublist:
        ingredients_set.add(item)
unique_ingredients = sorted(ingredients_set)  # list of sorted ingredients

In [5]:
# Save to a text file, one ingredient per line
with open("checkpoints/unique_ingredients.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(unique_ingredients))

print(f"Saved {len(unique_ingredients)} unique ingredients to unique_ingredients.txt")

Saved 1037 unique ingredients to unique_ingredients.txt
